In [42]:
import requests 
from datetime import date
import pandas as pd
import csv
# import pyspark.pandas as ps


In [43]:
def busca_artigo_keyword(url, chave_api, keyword):
    url_nova = url + '?q='+keyword+'&apiKey='+chave_api
    r = requests.get(url_nova)
    data = r.json()
    return data

def busca_artigo_keyword_publicacao_dia(url, chave_api, keyword):
    data_hoje = str(date.today())
    print(data_hoje)
    #url_nova = url + '?q='+keyword+'&apiKey='+chave_api+'&publishedAt>='+data_hoje
    url_nova = url + '?q='+keyword+'&apiKey='+chave_api+'&from='+data_hoje
    r = requests.get(url_nova)
    data = r.json()
    return data

### Primeira chamada
Traz todos os artigos com a palavra chave 'bitcoin'

In [44]:
chave_api = '657a3d08bfd94920bc9e527fac57ee28'
url = 'https://newsapi.org/v2/everything'

dados = busca_artigo_keyword(url, chave_api, 'bitcoin')
dados['articles'][0]


{'source': {'id': 'wired', 'name': 'Wired'},
 'author': 'Joel Khalili',
 'title': 'What’s Behind the Bitcoin Price Surge? Vibes, Mostly',
 'description': 'The price of bitcoin has climbed to a new all-time high. But assigning the cryptocurrency a value is anything but trivial.',
 'url': 'https://www.wired.com/story/bitcoin-price-record-economics/',
 'urlToImage': 'https://media.wired.com/photos/65ef46042ca08b0e59a9682f/191:100/w_1280,c_limit/031124-business-bitcoin-economics.jpg',
 'publishedAt': '2024-03-12T18:13:56Z',
 'content': 'The latest surge in the price of bitcoin is increasing the clamor around it, says Dal Bianco, drawing in yet more speculators and creating a self-reinforcing cycle. Likewise, when collective confiden… [+2967 chars]'}

Cria CSV com a primeira chamada, enviando o histórico das noticias

In [ ]:
articles = dados['articles']
df = pd.DataFrame(articles)
caminho_arquivo_csv = 'dados.csv'
df.to_csv(caminho_arquivo_csv, index=False)


## Demais chamadas 
Traz

In [19]:
dados_hoje = busca_artigo_keyword_publicacao_dia(url, chave_api, 'bitcoin')
dados_hoje

2024-04-01


{'status': 'ok',
 'totalResults': 3,
 'articles': [{'source': {'id': None, 'name': 'Erickimphotography.com'},
   'author': 'ERIC KIM',
   'title': 'Growth Opportunities',
   'description': 'How much room or opportunity is there for growth? With Bitcoin, sky’s the limit! I think with bitcoin, you could easily hit $150,000 a bitcoin, post halving, and then after that, I wouldn’t be surprised to see one bitcoin being worth $750,000, $1.3 million, $…',
   'url': 'https://erickimphotography.com/blog/2024/03/31/growth-opportunities/',
   'urlToImage': None,
   'publishedAt': '2024-04-01T00:20:59Z',
   'content': 'How much room or opportunity is there for growth?\r\nWith Bitcoin, skys the limit!\r\nI think with bitcoin, you could easily hit $150,000 a bitcoin, post halving, and then after that, I wouldnt be surpri… [+457 chars]'},
  {'source': {'id': None, 'name': 'Biztoc.com'},
   'author': 'dailyhodl.com',
   'title': 'Top Trader Predicts Fresh Leg Up for SOL and One Solana-Based Altcoin, U